- creating a session for pyspark

In [1]:
from pyspark.sql import SparkSession;
from pyspark.context import SparkContext
spark = SparkSession \
    .builder \
    .appName("pyspark basics") \
    .getOrCreate()

sc = spark.sparkContext

In [2]:
import pandas as pd
df = pd.read_excel('./data/LR.xlsx')
df.head(5)

,Attempt,Success,Bait,Place
0,1,0,Bread,Grandma's Pond
1,2,1,Bread,Grandma's Pond
2,3,1,Bread,Grandma's Pond
3,4,1,Bread,Grandma's Pond
4,5,1,Bread,Grandma's Pond


In [3]:
df = spark.createDataFrame(df)
df.show(5)

c:\Users\veera\anaconda3\envs\DEV\Lib\site-packages\pyspark\sql\pandas\conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


+-------+-------+-----+--------------+
|Attempt|Success| Bait|         Place|
+-------+-------+-----+--------------+
|      1|      0|Bread|Grandma's Pond|
|      2|      1|Bread|Grandma's Pond|
|      3|      1|Bread|Grandma's Pond|
|      4|      1|Bread|Grandma's Pond|
|      5|      1|Bread|Grandma's Pond|
+-------+-------+-----+--------------+
only showing top 5 rows



In [4]:
from pyspark.sql.functions import col, when, count

df.select(*(count(when(col(c).isNull(), c)).alias(c) for c in df.columns)).show()

+-------+-------+----+-----+
|Attempt|Success|Bait|Place|
+-------+-------+----+-----+
|      0|      0|   0|    0|
+-------+-------+----+-----+



In [5]:
df.describe().show()

+-------+-----------------+------------------+--------+-----------------+
|summary|          Attempt|           Success|    Bait|            Place|
+-------+-----------------+------------------+--------+-----------------+
|  count|               22|                22|      22|               22|
|   mean|             11.5|0.5909090909090909|    NULL|             NULL|
| stddev|6.493586579592718|0.5032362797401964|    NULL|             NULL|
|    min|                1|                 0|   Bread|   Grandma's Pond|
|    max|               22|                 1|Hot Dogs|Uncle Ron's Canal|
+-------+-----------------+------------------+--------+-----------------+



In [12]:
from pyspark.sql.functions import corr

df.select(corr(df['Attempt'], df['Success'])).show()

+----------------------+
|corr(Attempt, Success)|
+----------------------+
|  -0.08014691611446192|
+----------------------+



In [13]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import ChiSquareTest

data = [(0.0, Vectors.dense(0.5, 10.0)),
        (0.0, Vectors.dense(1.5, 20.0)),
        (1.0, Vectors.dense(1.5, 30.0)),
        (0.0, Vectors.dense(3.5, 30.0)),
        (0.0, Vectors.dense(3.5, 40.0)),
        (1.0, Vectors.dense(3.5, 40.0))]
df = spark.createDataFrame(data, ["label", "features"])

r = ChiSquareTest.test(df, "features", "label").head()



print("pValues: " + str(r.pValues))
print("degreesOfFreedom: " + str(r.degreesOfFreedom))
print("statistics: " + str(r.statistics))

pValues: [0.6872892787909721,0.6822703303362126]
degreesOfFreedom: [2, 3]
statistics: [0.75,1.5]
